# 🚀 DreamerV3 Ultimate Training - Gold Trading AI

**Notebook optimizado para entrenamientos largos sin desconexiones**

---

## ✅ Características:
- Anti-desconexión mejorado (múltiples métodos)
- Entrenamiento en disco LOCAL (evita límites de Google Drive)
- Copia automática a Drive al finalizar
- Checkpoints cada 50k steps

---

## ⚙️ Setup:
1. **GPU**: Runtime → Change runtime type → **GPU (A100 recomendado)**
2. **Ejecutar celdas en orden**
3. **IMPORTANTE**: Ejecuta el código anti-desconexión en la consola del navegador (F12)

## 0️⃣ ANTI-DESCONEXIÓN (MUY IMPORTANTE)

### Método 1: Código en Consola del Navegador (MÁS EFECTIVO)

1. Presiona **F12** para abrir DevTools
2. Ve a la pestaña **Console**
3. Pega este código y presiona Enter:

```javascript
function ClickConnect() {
    console.log('🔄 Keeping alive... ' + new Date().toLocaleTimeString());
    var connectBtn = document.querySelector('#top-toolbar > colab-connect-button');
    if (connectBtn && connectBtn.shadowRoot) {
        var innerBtn = connectBtn.shadowRoot.querySelector('#connect');
        if (innerBtn) innerBtn.click();
    }
    document.dispatchEvent(new MouseEvent('mousemove'));
}
setInterval(ClickConnect, 60000);
console.log('✅ Anti-desconexión activado!');
```

### Método 2: Ejecuta la siguiente celda (backup)

In [ ]:
import IPython
from IPython.display import display, Javascript

js_code = '''
function ClickConnect() {
    console.log("🔄 Keep-alive: " + new Date().toLocaleTimeString());
    var selectors = ["colab-connect-button", "#top-toolbar > colab-connect-button"];
    for (var i = 0; i < selectors.length; i++) {
        var btn = document.querySelector(selectors[i]);
        if (btn && btn.shadowRoot) {
            var innerBtn = btn.shadowRoot.querySelector("#connect");
            if (innerBtn) { innerBtn.click(); break; }
        }
    }
    document.dispatchEvent(new MouseEvent("mousemove", {bubbles: true}));
}
setInterval(ClickConnect, 45000);
ClickConnect();
console.log("✅ Anti-desconexión activado!");
'''
display(Javascript(js_code))
print("✅ Anti-desconexión activado!")
print("⚠️  También ejecuta el código en F12 → Console para máxima efectividad")

## 1️⃣ Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Copiar Proyecto a Disco LOCAL

**Evita límites de cuota de Google Drive**

In [ ]:
import os
import shutil

DRIVE_PATH = '/content/drive/MyDrive/XAUX'
LOCAL_PATH = '/content/XAUX_local'

if not os.path.exists(DRIVE_PATH):
    print(f"❌ ERROR: No existe: {DRIVE_PATH}")
    !ls /content/drive/MyDrive/
else:
    print(f"📂 Copiando a disco local...")
    if os.path.exists(LOCAL_PATH):
        shutil.rmtree(LOCAL_PATH)
    shutil.copytree(DRIVE_PATH, LOCAL_PATH)
    os.chdir(LOCAL_PATH)
    print(f"✅ Listo! Directorio: {os.getcwd()}")
    print(f"💡 Entrenamiento local = Sin límites de Drive")

## 3️⃣ Instalar Dependencias

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pandas numpy tqdm matplotlib seaborn yfinance
print("✅ Dependencias instaladas!")

## 4️⃣ Verificar GPU

In [ ]:
import torch

print(f"🖥️  PyTorch: {torch.__version__}")
print(f"🎮 CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {gpu_name} ({gpu_mem:.1f} GB)")
    device = 'cuda'
else:
    device = 'cpu'
print(f"✅ Device: {device}")

## 5️⃣ Configuración

In [ ]:
TRAINING_STEPS = 1_000_000
BATCH_SIZE = 128 if device == 'cuda' else 16

print(f"⚙️  DreamerV3 Config:")
print(f"   Steps: {TRAINING_STEPS:,}")
print(f"   Batch: {BATCH_SIZE}")
print(f"   Device: {device}")

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    h = TRAINING_STEPS/1e6 * (5 if 'A100' in gpu else 8 if 'V100' in gpu else 12)
    print(f"\n⏱️  ~{h:.1f} horas en {gpu}")

## 6️⃣ Entrenar DreamerV3

In [ ]:
!python train/train_ultimate_150.py --steps {TRAINING_STEPS} --device {device} --batch-size {BATCH_SIZE}

## 7️⃣ Copiar Resultados a Drive

In [ ]:
import shutil, os, glob

LOCAL = '/content/XAUX_local/train/dreamer_ultimate'
DRIVE = '/content/drive/MyDrive/XAUX/train/dreamer_ultimate'
os.makedirs(DRIVE, exist_ok=True)

print("📤 Copiando a Drive...")
for f in ['ultimate_150_xauusd_final.pt', 'ultimate_150_xauusd_metrics.json']:
    src = f'{LOCAL}/{f}'
    if os.path.exists(src):
        shutil.copy2(src, f'{DRIVE}/{f}')
        print(f"   ✅ {f}")

# Último checkpoint
cps = sorted(glob.glob(f'{LOCAL}/*.pt'))
if cps:
    shutil.copy2(cps[-1], f'{DRIVE}/{os.path.basename(cps[-1])}')
    print(f"   ✅ {os.path.basename(cps[-1])}")

print("\n🎉 Guardado en Drive!")

## 8️⃣ Ver Resultados

In [ ]:
import json, os

for p in ['/content/XAUX_local/train/dreamer_ultimate/ultimate_150_xauusd_metrics.json',
          '/content/drive/MyDrive/XAUX/train/dreamer_ultimate/ultimate_150_xauusd_metrics.json']:
    if os.path.exists(p):
        with open(p) as f: m = json.load(f)
        print("="*60)
        print("📊 RESULTADOS DREAMERV3")
        print("="*60)
        print(f"\nVALIDATION: {m['validation']['total_return']:.1f}% | Sharpe: {m['validation']['sharpe_ratio']:.2f}")
        print(f"TEST:       {m['test']['total_return']:.1f}% | Sharpe: {m['test']['sharpe_ratio']:.2f}")
        break
else:
    print("❌ Métricas no encontradas")

## 9️⃣ Resumir (si se interrumpe)

In [ ]:
import glob, os
for d in ['/content/XAUX_local/train/dreamer_ultimate', '/content/drive/MyDrive/XAUX/train/dreamer_ultimate']:
    if os.path.exists(d):
        cps = [c for c in sorted(glob.glob(f'{d}/*.pt')) if 'final' not in c]
        if cps:
            print(f"📁 Último checkpoint: {os.path.basename(cps[-1])}")
            print(f"\n🔄 Para resumir:")
            print(f"!python train/train_ultimate_150.py --steps {TRAINING_STEPS} --device {device} --resume {cps[-1]}")
            break

---
## 📝 Tips
1. Ejecuta JavaScript en F12 → Console
2. Mantén la pestaña visible
3. Revisa cada 1-2h por CAPTCHAs
4. Copia a Drive regularmente

🎉 **¡Listo!**